In [2]:
import folium
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame 
import fiona

In [3]:
sensor_points = pd.read_csv('./cleaned datasets/single_sensor_locations.csv')
sensor_points.drop(['location'], axis = 1, inplace = True)

In [4]:
sensor_points

,sensor_id,latitude,longitude
0,1,-37.813494,144.965153
1,2,-37.813807,144.965167
2,3,-37.811015,144.964295
3,4,-37.814880,144.966088
4,5,-37.818742,144.967877
...,...,...,...
61,62,-37.809965,144.962165
62,63,-37.813331,144.966756
63,64,-37.799495,144.957923
64,65,-37.815694,144.966806


# Map all sensors with 110 meter radius (.001 degree longitude)

In [8]:
# create a map
melbourne = folium.Map(location=[np.median(sensor_points['latitude'].tolist()),
                         np.median(sensor_points['longitude'].tolist())], tiles = "Stamen Toner",
                        zoom_start=14)
for row in sensor_points.iterrows():
    row_values = row[1]
    location = [row_values['latitude'], row_values['longitude']]
    popup = popup = '<strong>' + 'sensor ' + str(row_values['sensor_id']) + '</strong>'
    marker = folium.Marker(location = location, popup = popup)
    marker.add_to(melbourne)
    folium.Circle(location,radius=111).add_to(melbourne)
    
melbourne

# Function for mapping features in a specific sensor's radius

In [5]:
lights = pd.read_csv('./location features/features with locations/light_points.csv')
bikes = pd.read_csv('./location features/features with locations/bike_points.csv')
furniture = pd.read_csv('./location features/features with locations/furniture_points.csv')
landmarks = pd.read_csv('./location features/features with locations/landmark_points.csv')
buildings = pd.read_csv('./location features/features with locations/building_points_2018.csv')
buildings

,point,building_use,n_floors,longitude,latitude,sensor_id
0,POINT (-37.8131 144.965),Commercial Accommodation,16,-37.8131,144.9650,1
1,POINT (-37.8142 144.9652),Retail - Shop,6,-37.8142,144.9652,1
2,POINT (-37.8137 144.966),Storage,3,-37.8137,144.9660,1
3,POINT (-37.8137 144.966),Retail - Shop,3,-37.8137,144.9660,1
4,POINT (-37.8133 144.9648),Retail - Shop,5,-37.8133,144.9648,1
...,...,...,...,...,...,...
1275,POINT (-37.8113 144.964),Office,4,-37.8113,144.9640,66
1276,POINT (-37.8106 144.9635),Office,2,-37.8106,144.9635,66
1277,POINT (-37.8115 144.9641),Unoccupied - Unused,4,-37.8115,144.9641,66
1278,POINT (-37.8109 144.9641),Entertainment/Recreation - Indoor,2,-37.8109,144.9641,66


In [10]:
def map_nearby_features(sensor_num):
    
    #set up base map using location points from the sensor
    sensor = sensor_points[sensor_points.sensor_id == sensor_num]
    
    sensor_map = folium.Map(location=[np.median(sensor['latitude'].tolist()),
                         np.median(sensor['longitude'].tolist())], tiles = "Stamen Toner",
                        zoom_start=20)

    title_html = '''
         <head><style> html { overflow-y: hidden; } </style></head>
         <h3 align="center" style="font-size:20px"><b>Nearby Features</b></h3>
         '''

    sensor_map.get_root().html.add_child(folium.Element(title_html))


    #add sensor point and radius
    for row in sensor.iterrows():
        row_values = row[1]
        location = [row_values['latitude'], row_values['longitude']]
        popup = popup = '<strong>' + 'sensor ' + str(row_values['sensor_id']) + '</strong>'
        marker = folium.Marker(location = location, popup = popup, icon=folium.Icon(color='darkblue', icon = 'star', prefix = 'fa'))
        marker.add_to(sensor_map)
        folium.Circle(location,radius=111, color = 'darkblue').add_to(sensor_map)
        
        
    #add lighting features nearby
    sensor_lights = lights[lights.sensor_id == sensor_num]

    for row in sensor_lights.iterrows():
        row_values = row[1]
        location = [row_values['latitude'], row_values['longitude']]
        popup = popup = '<strong>' + 'lighting ' + '</strong>'
        marker = folium.Marker(location = location, popup = popup, icon=folium.Icon(color='lightred', icon = 'fa-lightbulb-o', prefix = 'fa'))
        marker.add_to(sensor_map)
        
    #add buildings nearby  
    sensor_buildings = buildings[buildings.sensor_id == sensor_num]

    for row in sensor_buildings.iterrows():
        row_values = row[1]
        if row_values['building_use'] == 'Retail - Shop':
            location = [row_values['longitude'], row_values['latitude']]
            popup = popup = '<strong>' + 'building: ' +  '</strong>' + row_values['building_use'] + '<strong>' + ' number_floors: ' +  '</strong>' + ' ' + str(row_values['n_floors']) 
            marker = folium.Marker(location = location, popup = popup, icon=folium.Icon(icon = 'glyphicon glyphicon-tags', color='lightblue')).add_to(sensor_map)

        elif row_values['building_use'] == 'Office':
            location = [row_values['longitude'], row_values['latitude']]
            popup = popup = '<strong>' + 'building: ' +  '</strong>' + row_values['building_use'] + '<strong>' + ' number_floors: ' +  '</strong>' + ' ' + str(row_values['n_floors']) 
            marker = folium.Marker(location = location, popup = popup, icon=folium.Icon(icon = 'briefcase', prefix = 'fa', color='lightblue')).add_to(sensor_map)

        else:
            location = [row_values['longitude'], row_values['latitude']]
            popup = popup = '<strong>' + 'building: ' +  '</strong>' + row_values['building_use'] + '<strong>' + ' number_floors: ' +  '</strong>' + ' ' + str(row_values['n_floors']) 
            marker = folium.Marker(location = location, popup = popup, icon=folium.Icon(icon = 'building', prefix = 'fa', color='lightblue')).add_to(sensor_map)

    #add landmarks nearby
    sensor_landmarks = landmarks[landmarks.sensor_id == sensor_num]

    for row in sensor_landmarks.iterrows():
        row_values = row[1]
        location = [row_values['longitude'], row_values['latitude']]
        popup = popup = '<strong>' + row_values['theme'] + '</strong>'
        marker = folium.Marker(location = location, popup = popup, icon=folium.Icon(color='purple', icon = 'group', prefix = 'fa'))
        marker.add_to(sensor_map)

    #add furniture nearby
    sensor_furniture = furniture[furniture.sensor_id == sensor_num]
    
    for row in sensor_furniture.iterrows():
        row_values = row[1]
        if row_values['feature'] == 'Seat':
            location = [row_values['longitude'], row_values['latitude']]
            popup = popup = '<strong>' + row_values['feature'] + '</strong>'
            marker = folium.Marker(location = location, popup = popup, icon=folium.Icon(color='lightgray', icon = 'map-signs', prefix = 'fa')).add_to(sensor_map)
        if row_values['feature'] == 'Bicycle Rails':
            location = [row_values['longitude'], row_values['latitude']]
            popup = popup = '<strong>' + row_values['feature'] + '</strong>'
            marker = folium.Marker(location = location, popup = popup, icon=folium.Icon(color='lightgray', icon = 'bicycle', prefix = 'fa')).add_to(sensor_map)
        if row_values['feature'] == 'Litter Bin':
            location = [row_values['longitude'], row_values['latitude']]
            popup = popup = '<strong>' + row_values['feature'] + '</strong>'
            marker = folium.Marker(location = location, popup = popup, icon=folium.Icon(color='lightgray', icon = 'trash', prefix = 'fa')).add_to(sensor_map)
        else: 
            location = [row_values['longitude'], row_values['latitude']]
            popup = popup = '<strong>' + row_values['feature'] + '</strong>'
            marker = folium.Marker(location = location, popup = popup, icon=folium.Icon(color='lightgray', icon = 'map-marker', prefix = 'fa')).add_to(sensor_map)
            
            
    #add bike share docks nearby
    sensor_bikes = bikes[bikes.sensor_id == sensor_num]

    for row in sensor_bikes.iterrows():
        row_values = row[1]
        location = [row_values['latitude'], row_values['longitude']]
        popup = popup = '<strong>' + 'bike dock ' + '</strong>'
        marker = folium.Marker(location = location, popup = popup, icon=folium.Icon(color='lightgreen', icon = 'bicycle', prefix = 'fa'))
        marker.add_to(sensor_map)
        
    return sensor_map

In [11]:
map_nearby_features(28)